In [1]:
import requests
import pandas as pd
from pandas.io.json import json_normalize

## Reading Freshsales CRM API

In [ ]:
import requests
import pandas as pd
from pandas.io.json import json_normalize

token = 'enter here your api key'
url = 'https://your-domain.freshsales.io/deals/view/4000231473?include=deal_stage'   # All Deals

response = requests.get(url, \
                        headers={'Authorization': 'Token token='token, \
                                 'Content-Type': 'application/json'})

# Extract deal_stages and Deals from JSON response in separate 
deal_stages =  json_normalize(response.json(), 'deal_stages')
deals = json_normalize(response.json(), 'deals')

# Resolve dictionaries in columns custom_field and collaboration
df_custom_fields = deals['custom_field'].apply(pd.Series)
df_collaboration = deals['collaboration'].apply(pd.Series)
                                 
# Concatenate along columns (axis = 1) and drop old unresolved column
deals = pd.concat([deals, df_custom_fields], axis = 1).drop('custom_field', axis=1)
deals = pd.concat([deals, df_collaboration], axis = 1).drop('collaboration', axis=1)
deals = deals[['id', 'name','amount', 'base_currency_amount', 'deal_stage_id','cf_expected_start_date','cf_project_duration_in_months']]

# Some parsing and transformations
deals['cf_project_duration_in_months'] = deals['cf_project_duration_in_months'].fillna(0).astype(int)
deals['cf_expected_start_date'] = pd.to_datetime(deals['cf_expected_start_date'])
deals['start_date'] = pd.to_datetime(deals['cf_expected_start_date'].dt.strftime('%Y/%m/%d')) 
deals = deals.iloc[:,~deals.columns.duplicated()]

# Merge deals with Deal stages again and create new DataFrame df
df = pd.merge(deals, deal_stages, left_on = 'deal_stage_id', right_on = 'id')
df = df[['id_x', 'name_x','amount', 'base_currency_amount', 'deal_stage_id', 'name_y','start_date','cf_project_duration_in_months']]
df = df.rename(columns={'name_x': 'deal_name', 'id_x': 'deal_id', 'name_y': 'deal_stage_name', \
                        'cf_project_duration_in_months': 'duration', \
                        'amount': 'total_amount'})

df['total_amount'] = df['total_amount'].astype('float')

# Enables custom deal_stage sort in PowerBI later on
# Enter here your deal stagesa and adapt dictionary correspondingly
dict = {'Stage 1' : 1, 'Stage 2': 2, 'Stage 3': 3, 'Stage 4': 4, 'Stage 5': 5}
df['custom_deal_id'] = df.deal_stage_name.map(dict)


################-------------------Transformations for Forecast-------------------##################

                                 
# Create two DataFrames out of API Result (=df)
# ff_false - contains rows where a calculation Forecast is not possible
# df_true - Forecast possible
df_false = df[df['duration'] == 0].reset_index()
df_true = df[df['duration'] != 0].reset_index()
df_true['avg_amount'] = df_true['total_amount']
df_true['period'] = df_true['start_date']

# Create, Transform and Perform linear Forecast of deal value(amount)
df_true2 = pd.DataFrame(columns = df_true.columns)

for index, row in df_true.iterrows():
   
    start_date = df_true.iloc[index]['start_date']
    duration = df_true.iloc[index]['duration']
    amount = df_true.iloc[index]['total_amount']
    avg_amount = amount/duration
    
    for i in range(duration):
        start_date = start_date + pd.DateOffset(months=1)
        row[11] = start_date
        row[10] = avg_amount
        df_true2 = df_true2.append([[row][0]])

df_true2 = df_true2.reset_index().drop(['level_0','index'], axis=1)
df_true2['period2'] = pd.DatetimeIndex(df_true2['period']).to_period('M')

# Concatenate both DataFrames 
AllDealsDF = pd.concat([df_false, df_true2], axis = 0, sort=True)
AllDealsDF.reset_index(inplace = True) 
AllDealsDF.drop(['level_0', 'index'], axis = 1, inplace = True)
AllDealsDF = round(AllDealsDF,2)
AllDealsDF.head() # Result of all Operations is DataFrame AllDealsDF

## Freshsales  Writing Deals

In [6]:
# To Do: Fix accout status and region

import requests
import json

token = 'enter here your API Key'
url = 'https://your-domain.freshsales.io/deals'

deal = {"deal":
                {
                 "name":"Sample deal",
                 "amount":23456,
                 "sales_account_id":4000235827,
                 "deal_payment_status_id": None,
                 'contact_ids': 4001954545,
                 "deal_stage_id" : 4000045174,
                 "deal_pipeline_id" : 4000006256,
                 "owner_id":1,
                 "lead_source_id":1,
                 "deal_product_id" : 1,
                 "Deal source": 212,
                 "email": "blabla@web.de",
                 "Deal Owner" : "Your Accunt name",
                 "Related contacts" : 1,
                 'custom_field': {'cf_existing_business': 'New Project',
                                  'cf_sub-domain': '###',
                                  'cf_products': '###'},
                 'title': 'Test title',
                 "Deal Source" : "Email",
              
                }
       }

r = requests.post(url, \
                  headers={'Authorization': 'Token token='token, \
                            'Content-Type': 'application/json'}, \
                  data = json.dumps(deal)     
                 )

if r.status_code == 201:
  print("Deal created successfully, the response is given below", r.content)
  print("Location Header : " , r.headers['Location'])
else:
  print("Failed to create deal, errors are displayed below")
  response = json.loads(r.content)
  print("")
  print(response["errors"])

print("")
print("x-request-id : " , r.headers['x-request-id'])
print("Status Code : " , str(r.status_code))

Failed to create deal, errors are displayed below

{'code': 400, 'message': ["Account status can't be empty", "Movilitas region can't be empty"]}

x-request-id :  22848f4a88dc418b13eb587ae1d00275
Status Code :  400
